In [ ]:
#Librerías
import gurobipy as gp
from gurobipy import GRB

#PARÁMETROS DEL MODELO

**n** = número de jobs

**m** = número de máquinas

**a_i,j,h** = indica si la operación O_j,h puede realizarse en la máquina "i" (1 = si, 0 = no)

**P_i,j,h**= tiempo de procesamiento si O_j, se ejecuta en la máquina "i"

**L** = un número muy grande, para restricciones con Big-M

#VARIABLES DEL MODELO

**y[i,j,o]**: si la operación "o" del trabajo "j" se asigna a la máquina "i"
    Es la desición de asginación de operacioens a máquinas.

**x[i,j,o,k]**: si la operacion "o" del trabajo "j" ocupa la posición "k" en la máquina "i"
    Permite modelar el orden de ejecución (secuencia) dentro de cada máquina.

**t[j,h]**: Tiempo de inicio del procesamiento de la operación O_j,h

**Tm_i,k**: El instante de tiempo en que la máquina "i" empieza a trabajar en su k-ésima operación.

**k_i**: Número de operaciones asignadas a la máquina "i".

**Ps[j,h]**: Tiempo de procesa,miento de la operación O_j,h luego de ser asignada a una máquina.

**Cmax**: Tiempo total de finalización del último trabajo.

*x e y son variables Binarias; t, T, Ps y Cmax son variables Continuas.*

#RESTRICCIONES (por definir):

**1:** El tiempo total del sistema debe ser mayor o igual al término de cada trabajo individual.

**2:** “selecciona el tiempo de procesamiento correcto” según la máquina elegida.

**3:** La operación h + 1 no puede empezar antes de que la operación h haya terminado

**4:** El inicio del slot siguiente debe ser después de que termine la operación que está en el slot actual. (Las máquinas tienen slots)

**5:** Hace coincidir el reloj de la máquina con el inicio de la operación asignada (Cada máquina tiene su propio “tiempo interno”, Tm_i,k)

**6:** El inicio del slot debe ser igual o después del inicio real de la operación que se ejecuta ahí.

** 5 y 6 juntas:** El modelo sincroniza los relojes de la máquina con los inicios de las operaciones que realmente se ejecutan ahí.

**7:** Impide asignar una operación a una máquina que no sea capaz de ejecutarla.

**8:** Cada posición (slot) en cada máquina tenga exactamente una operación asignada.

**9:** Exactamente una máquina por operación

**10:** Si una operación j,h se asigna a una máquina i, entonces debe ocupar exactamente un slot k de esa máquina. Si una operación no está asignada a la máquina i,
entonces no puede aparecer en ningún slot de esa máquina.

**11:** El tiempo de inicio de cada operación j,h no puede ser negativo.

**12:** No negatividad del tiempo de proceso efectivo

**13:** no negatividad del “reloj” de la máquina. Fue inicializada con lowerbound = 0, asi que la restricción es redundante (pero se mantendrá)

**14:** x_i,j,h,k es 0 o 1

**15:** y_i,j,h es 0 o 1

In [ ]:
def solve_fjsp(data, timelimit=3600):

    #Parámetros:

    J = data['J']           # lista de trabajos
    M = data['M']           # lista de máquinas
    H = data['H']           # operaciones por trabajo (dict)
    a = data['a']       # máquinas capaces
    p = data['p']           # tiempos p[i,j,h]
    K = {i: sum(H[j] for j in J) for i in M}  # cota superior para k_i
    BIGM = sum(max(p[(i,j,h)] for i in M if a[(i,j,h)] == 1) for j in J for h in range(H[j]))



    #Modelo:

    model = gp.Model("FJSP_Fattahi_Tarea_3") #instancia del modelo
    model.Params.TimeLimit = timelimit #define límite de tiempo



    #Variables:
    y = model.addVars([(i,j,h) for i in M for j in J for h in range(H[j])],
                  vtype=GRB.BINARY, name="y")
    x = model.addVars([(i,j,h,k) for i in M for j in J for h in range(H[j]) for k in range(K[i])],
                      vtype=GRB.BINARY, name="x")
    t = model.addVars([(j,h) for j in J for h in range(H[j])], lb=0.0, name="t")
    Tm = model.addVars([(i,k) for i in M for k in range(K[i])], lb=0.0, name="Tm")
    Ps = model.addVars([(j,h) for j in J for h in range(H[j])], lb=0.0, name="Ps")
    Cmax = model.addVar(lb=0.0, name="Cmax")



    #Función Objetivo:
    model.setObjective(Cmax, GRB.MINIMIZE)




    #Restricciones:

    #(1): Cmax mayor o igual que el final de cada trabajo
    for j in J:
        last = H[j] - 1
        model.addConstr(Cmax >= t[j,last] + Ps[j,last], name=f"cmax_{j}")




    for j in J:
        for h in range(H[j]):
            #(2):
            model.addConstr(
                Ps[j,h] == gp.quicksum(p[(i,j,h)] * y[i,j,h] for i in M),
                name=f"ptime_{j}_{h}"
                )
            #(9):
            model.addConstr(
                gp.quicksum(y[i,j,h] for i in M) == 1,
                name=f"one_machine_per_operation_{j}_{h}"
            )
            #(11):
            model.addConstr(
                t[j,h] >= 0,
                name=f"start_time_nonnegative_{j}_{h}"
                )
            #(12):
            model.addConstr(
                Ps[j,h] >= 0,
                name=f"processing_time_nonnegative_{j}_{h}"
            )

        for h in range(H[j]-1):
            #(3):
            model.addConstr(t[j,h] + Ps[j,h] <= t[j,h+1], name=f"prec_{j}_{h}")










    for i in M:
        for j in J:
            for h in range(H[j]):
                #(7):
                model.addConstr(
                y[i,j,h] <= a[(i,j,h)],
                name=f"machine_capability_{i}_{j}_{h}"
                )
                #(10):
                model.addConstr(
                    gp.quicksum(x[i,j,h,k] for k in range(K[i])) == y[i,j,h],
                    name=f"link_y_x_{i}_{j}_{h}"
                )
                #Parece que las restricciones (15) no son necesarias al definir y como variable binaria,
                #pero estarían acá.


                for k in range(K[i] - 1):
                    #(4)
                    model.addConstr(
                        Tm[i,k] + Ps[j,h] * x[i,j,h,k] <= Tm[i, k + 1],
                        name=f"slot_link_{i}_{j}_{h}_{k}"
                    )

                for k in range (K[i]):
                    #(5)
                    model.addConstr(
                        Tm[i,k] <= t[j,h] + (1 - x[i,j,h,k]) * BIGM,
                        name=f"start_time_link_{i}_{j}_{h}_{k}"
                    )
                    #(6)
                    model.addConstr(
                        Tm[i,k] + (1 - x[i,j,h,k]) * BIGM >= t[j,h],
                        name=f"start_time_link2_{i}_{j}_{h}_{k}"
                    )

                    #Parece que las restricciones (14) no son necesarias al definir x como variable binaria,
                    #pero estarían acá.


    for i in M:
        for k in range(K[i]):
            #(8): gpt dice que debe ser <= 1, pero el paper dice que es = 1
            model.addConstr(
                gp.quicksum(x[i,j,h,k] for j in J for h in range(H[j])) == 1,
                name=f"one_operation_per_slot_{i}_{k}"

            )

            #(13):
            model.addConstr(
                Tm[i,k] >= 0,
                name=f"machine_time_nonnegative_{i}_{k}"
            )











